<h3>Run spark only from terminal!!!</h3>

source activate spark

jupyter-lab

In [1]:
%%script false
import os
#os.environ["JAVA_HOME"]

In [2]:
%%script false
import os

os.environ["JAVA_HOME"] = r"/usr/lib/jvm/java-8-oracle"
os.environ["SPARK_HOME"] = r"/home/ubuntu/spark"
import sys
sys.path.insert(0, os.environ["SPARK_HOME"])


os.environ["PYSPARK_PYTHON"] = r"/home/batman/miniconda3/envs/spark/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"/home/batman/miniconda3/envs/spark/bin/python"

for i in os.environ:
    print(i, "=", os.environ[i])
    
#os.environ["PATH"].append()

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]') # asdf

In [4]:
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


Sample output

In [5]:
fname = "macbeth.txt"

with open(fname) as f:
    my_lines = f.readlines()

#for l in my_lines:
#    print(l)

print(len(my_lines))

for l in my_lines[:5]:
    print(l)



124787
﻿The Project Gutenberg EBook of The Complete Works of William Shakespeare, by

William Shakespeare



This eBook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever.  You may copy it, give it away or



In [6]:
#!script = falsse
# load text of macbeth from s3
#!wget https://dsr-notebooks.s3.amazonaws.com/macbeth.txt
shakespeare = sc.textFile("macbeth.txt")

print(shakespeare)

print(type(shakespeare))

macbeth.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0
<class 'pyspark.rdd.RDD'>


In [7]:
lines_all = shakespeare.map(lambda line: line.lower())
print(lines_all)
#print(len(lines))
print(type(lines_all))

PythonRDD[2] at RDD at PythonRDD.scala:48
<class 'pyspark.rdd.PipelinedRDD'>


In [8]:
shakespeare.take(5)

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or']

In [9]:
shakespeare.count()

124787

Now, this is stored in memory

In [10]:
lines_all.cache()

PythonRDD[2] at RDD at PythonRDD.scala:48

Find the lines containing 'macbeth'

In [11]:
lines_macbeth = lines_all.filter(lambda l: l.find("macbeth") >= 0)
#import sys
print(lines_macbeth)

PythonRDD[5] at RDD at PythonRDD.scala:48


In [12]:
lines_macbeth.count()

283

In [13]:
lines_macbeth.take(5)

['the tragedy of macbeth',
 "  macbeth, thane of glamis and cawdor, a general in the king's army",
 '  lady macbeth, his wife',
 '  seyton, attendant to macbeth',
 '  gentlewoman attending on lady macbeth']

In [21]:
#lines_macbeth_and_macduff = lines_all.filter(lambda l: l.find("macduff") >= 0)
lines_macbeth_and_macduff = lines_macbeth.filter(lambda l: l.find("macduff") >= 0)

In [22]:
print(lines_macbeth_and_macduff)
lines_macbeth_and_macduff.collect()

PythonRDD[11] at RDD at PythonRDD.scala:48


['  macduff. those that macbeth hath slain.',
 "  macbeth. how say'st thou, that macduff denies his person",
 '  first apparition. macbeth! macbeth! macbeth! beware macduff,',
 '  macbeth. then live, macduff. what need i fear of thee?',
 "             re-enter macduff, with macbeth's head."]

In [23]:
print(lines_macbeth_and_macduff.toDebugString())
#print(macbeth_and_macdufflines.toDebugString())

b'(2) PythonRDD[11] at RDD at PythonRDD.scala:48 []\n |  PythonRDD[2] at RDD at PythonRDD.scala:48 []\n |      CachedPartitions: 2; MemorySize: 4.0 MB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B\n |  macbeth.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []\n |  macbeth.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []'


Get all words

In [24]:
import re
words = lines_all.flatMap(lambda l: re.split(r'\W+', l))
words.take(3)

['the', 'project', 'gutenberg']

Convert each word into a tuple

In [27]:
groupedWords = words.map(lambda w: (w, 1))
print(groupedWords)
groupedWords.take(3)

PythonRDD[16] at RDD at PythonRDD.scala:48


[('the', 1), ('project', 1), ('gutenberg', 1)]

Now, sum the counts (reduce)

In [30]:
unsorted = groupedWords.reduceByKey(lambda a, b: a + b)
print(unsorted)
print(unsorted.take(3))

PythonRDD[33] at RDD at PythonRDD.scala:48
[('project', 331), ('gutenberg', 326), ('ebook', 17)]


In [33]:
print(unsorted.collect)
values = unsorted.collect()[:10]
for v in values:
    print(v)

<bound method RDD.collect of PythonRDD[33] at RDD at PythonRDD.scala:48>
('project', 331)
('gutenberg', 326)
('ebook', 17)
('of', 18307)
('shakespeare', 272)
('', 197060)
('this', 6900)
('is', 9808)
('use', 562)
('anyone', 7)


All in one

In [37]:
swapped_unsorted = unsorted.map(lambda t: (t[1], t[0]))
print(swapped_unsorted)
print(swapped_unsorted.take(3))

PythonRDD[43] at RDD at PythonRDD.scala:48
[(331, 'project'), (326, 'gutenberg'), (17, 'ebook')]


In [38]:
swapped_unsorted.sortByKey(False).collect()[:10]

[(197060, ''),
 (27843, 'the'),
 (26847, 'and'),
 (22538, 'i'),
 (19882, 'to'),
 (18307, 'of'),
 (14800, 'a'),
 (13928, 'you'),
 (12490, 'my'),
 (11563, 'that')]

In [39]:
sc.stop()